<a href="https://colab.research.google.com/github/mithunkumarsr/NLPMay22/blob/main/Yelp_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch torchvision torchaudio


     |████████████████████████████████| 1.9 MB 3.1 MB/s 


In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 2.8 MB 3.2 MB/s 
     |████████████████████████████████| 895 kB 66.4 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 636 kB 57.8 MB/s 
     |████████████████████████████████| 3.3 MB 33.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')


In [ ]:
result = model(tokens)
print(result)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


In [ ]:
result.logits


tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>)

In [ ]:
int(torch.argmax(result.logits))+1


4

In [ ]:

r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

['Still one of the favorite coffee shop in Sydney. Staffs have excellent knowledge about beans , flavor, brew skills. They make own pastries which are also tasty too.',
 'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.',
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!",
 'Good coffee and toasts. Straight up and down - hits the spot with nothing mind blowing. Solid and tasty. \xa0Good work',
 "Ron & Jo are on the go down under and Wow! \xa0We've found our breakfast place in Sydney and are coming bac

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [ ]:
df['review'].iloc[0]


'Still one of the favorite coffee shop in Sydney. Staffs have excellent knowledge about beans , flavor, brew skills. They make own pastries which are also tasty too.'

In [ ]:

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])


5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))


In [ ]:
df

,review,sentiment
0,Still one of the favorite coffee shop in Sydne...,5
1,I came to Social brew cafe for brunch while ex...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,Good coffee and toasts. Straight up and down -...,5
4,Ron & Jo are on the go down under and Wow! We...,5
5,We came for brunch twice in our week-long visi...,4
6,Great coffee and vibe. That's all you need. C...,5
7,Great coffee and vibe. That's all you need. C...,4
8,This is my fave brunch café in and around Sydn...,5
9,This cafe is one of the most popular cafes whe...,5
